<div style="background-color: #058212; border-radius: 20px; color: #fff; font-weight: bold; padding: 10px; text-align:center">
    <h1>EDA para dataset de ventas de e-commerce</h1>
</div>

<div style="font-weight: bold; color:#058212 ; border-width: 0 0 3px 0; border-style: solid; border-color: #058212; padding: 3px; ">
    <h2>Cargar librerías</h2>
</div>

In [1]:
import pandas as pd
import numpy as np

<div style="font-weight: bold; color:#058212 ; border-width: 0 0 3px 0; border-style: solid; border-color: #058212; padding: 3px; ">
    <h2>Carga y configuración inicial</h2>
</div>

In [2]:
np.random.seed(42)
n_orders = 1000

df = pd.DataFrame({
    'order_id': range(1, n_orders + 1),
    'customer_id': np.random.randint(1, 201, n_orders),
    'product_id': np.random.randint(1, 51, n_orders),
    'quantity': np.random.randint(1, 5, n_orders),
    'unit_price': np.round(np.random.uniform(10, 500, n_orders), 2),
    'order_date': pd.date_range('2023-01-01', periods=n_orders, freq='h')[:n_orders],
    'payment_method': np.random.choice(['Credit Card', 'Debit Card', 'PayPal', 'Cash'], n_orders),
    'customer_age': np.random.normal(35, 10, n_orders).clip(18, 80).astype(int),
    'shipping_region': np.random.choice(['North', 'South', 'East', 'West'], n_orders)
})

# Introducir algunos valores faltantes
mask = np.random.random(n_orders) < 0.05
df.loc[mask, 'customer_age'] = np.nan

print("Dataset cargado exitosamente")

Dataset cargado exitosamente


<div style="font-weight: bold; color:#058212 ; border-width: 0 0 3px 0; border-style: solid; border-color: #058212; padding: 3px; ">
    <h2>Inspección inicial sistemática</h2>
</div>

In [3]:
# Dimensiones y estructura
print(f"Dataset shape: {df.shape}")
print(f"\nColumnas: {list(df.columns)}")
print(f"\nTipos de datos:\n{df.dtypes}")

# Primeras y últimas filas
print("\nPrimeras 5 filas:")
print(df.head())

print("\nÚltimas 5 filas:")
print(df.tail())

Dataset shape: (1000, 9)

Columnas: ['order_id', 'customer_id', 'product_id', 'quantity', 'unit_price', 'order_date', 'payment_method', 'customer_age', 'shipping_region']

Tipos de datos:
order_id                    int64
customer_id                 int32
product_id                  int32
quantity                    int32
unit_price                float64
order_date         datetime64[ns]
payment_method             object
customer_age              float64
shipping_region            object
dtype: object

Primeras 5 filas:
   order_id  customer_id  product_id  quantity  unit_price  \
0         1          103          40         2      371.34   
1         2          180          49         2      260.99   
2         3           93          44         3      343.31   
3         4           15          19         2       30.42   
4         5          107          42         2       51.55   

           order_date payment_method  customer_age shipping_region  
0 2023-01-01 00:00:00     Debit

<div style="font-weight: bold; color:#058212 ; border-width: 0 0 3px 0; border-style: solid; border-color: #058212; padding: 3px; ">
    <h2>Análisis de calidad de datos</h2>
</div>

In [4]:
# Valores faltantes
print("Valores faltantes por columna:")
print(df.isnull().sum())

print(f"\nPorcentaje de completitud: {(1 - df.isnull().sum() / len(df)) * 100}")

# Valores únicos por columna
print("\nValores únicos por columna:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].nunique()} valores únicos")

# Estadísticos básicos para numéricas
print("\nEstadísticos básicos de variables numéricas:")
print(df.select_dtypes(include=[np.number]).describe())

Valores faltantes por columna:
order_id            0
customer_id         0
product_id          0
quantity            0
unit_price          0
order_date          0
payment_method      0
customer_age       46
shipping_region     0
dtype: int64

Porcentaje de completitud: order_id           100.0
customer_id        100.0
product_id         100.0
quantity           100.0
unit_price         100.0
order_date         100.0
payment_method     100.0
customer_age        95.4
shipping_region    100.0
dtype: float64

Valores únicos por columna:
payment_method: 4 valores únicos
shipping_region: 4 valores únicos

Estadísticos básicos de variables numéricas:
          order_id  customer_id   product_id     quantity   unit_price  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean    500.500000   101.697000    25.680000     2.469000   253.828640   
std     288.819436    57.629249    14.180645     1.121739   141.648562   
min       1.000000     1.000000     1.000000     1.0

order_id, customer_id, product_id son variables categóricas nominales, por lo que carece de sentido calcular los estadísticos descriptivos adecuados para variables numéricas, por lo que su análisis debe basarse en frecuencias y/o conteos.

Con ello, los estadísticos básicos para las variables numéricas del dataset serían:

In [5]:
print("\nEstadísticos básicos de variables numéricas:")
print(df[['quantity', 'unit_price', 'customer_age']].describe())


Estadísticos básicos de variables numéricas:
          quantity   unit_price  customer_age
count  1000.000000  1000.000000    954.000000
mean      2.469000   253.828640     34.694969
std       1.121739   141.648562      9.615823
min       1.000000    10.070000     18.000000
25%       1.000000   137.067500     28.000000
50%       2.000000   254.675000     34.000000
75%       3.000000   377.210000     41.000000
max       4.000000   499.190000     65.000000


<div style="font-weight: bold; color:#058212 ; border-width: 0 0 3px 0; border-style: solid; border-color: #058212; padding: 3px; ">
    <h2>Preguntas exploratorias iniciales</h2>
</div>

In [6]:
# Distribución por región
print("Distribución de pedidos por región:")
print(df['shipping_region'].value_counts())

# Método de pago más popular
print(f"\nMétodo de pago más usado: {df['payment_method'].value_counts().index[0]}")

# Rango de fechas
print(f"\nPeríodo de datos: {df['order_date'].min()} a {df['order_date'].max()}")

# Edad promedio de clientes
edad_promedio = df['customer_age'].mean()
print(f"\nEdad promedio de clientes: {edad_promedio:.1f} años")

Distribución de pedidos por región:
shipping_region
South    267
North    250
East     246
West     237
Name: count, dtype: int64

Método de pago más usado: Credit Card

Período de datos: 2023-01-01 00:00:00 a 2023-02-11 15:00:00

Edad promedio de clientes: 34.7 años


En este caso hay solo una moda (por construcción del dataset), pero para calcular el o los métodos de pago más usados deberíamos obtener todos aquellos métodos cuya frecuencia coincide con el valor máximo.

In [7]:
mask = df['payment_method'].value_counts()==df['payment_method'].value_counts().max()
print(f"Método(s) de pago más usado(s): {df['payment_method'].value_counts()[mask].index.tolist()}")

Método(s) de pago más usado(s): ['Credit Card']


Otras preguntas iniciales podrían ser:
- top 3 productos más vendidos,
- top 3 productos más vendidos por región, 
- top 3 productos que reportan los mayores ingresos por región, etc.

In [8]:
# top 3 productos más vendidos
# obtener todos los productos con frecuencia mayor o igual a la tercera mayor frecuencia

def obtener_top_k(df, col, k=3):
    '''
    Calcula la suma total para la columna 'col' por 'product_id', ordena y devuelve
    los top k productos, incluyendo empates en el límite.
    '''
    total_agrupado = df.groupby('product_id')[col].sum().sort_values(ascending=False)
    valor_limite = total_agrupado.iloc[k-1]
    mask = total_agrupado >= valor_limite
    return total_agrupado[mask]

top_3 = obtener_top_k(df, col='quantity', k=3)
print(f'Los ids de los top {len(top_3)} productos más vendidos son: {top_3.index.to_list()}')

print(f'\nEn este caso hay 4 productos porque hay dos que comparten la misma frecuencia:\n{top_3}')

Los ids de los top 4 productos más vendidos son: [29, 37, 30, 33]

En este caso hay 4 productos porque hay dos que comparten la misma frecuencia:
product_id
29    77
37    70
30    67
33    67
Name: quantity, dtype: int32


In [9]:
# top 3 productos más vendidos por región

for region in sorted(df.shipping_region.unique()):
    top_3_region = obtener_top_k(df[df.shipping_region==region], col='quantity', k=3)
    print(f'Los ids de los top {len(top_3_region)} productos más vendidos para la región {region} son: {top_3_region.index.to_list()}')

Los ids de los top 3 productos más vendidos para la región East son: [40, 30, 25]
Los ids de los top 3 productos más vendidos para la región North son: [6, 7, 49]
Los ids de los top 3 productos más vendidos para la región South son: [35, 38, 30]
Los ids de los top 3 productos más vendidos para la región West son: [37, 34, 22]


In [10]:
# top 3 productos que reportan los mayores ingresos por región

df['ingresos'] = df.quantity * df.unit_price

for region in sorted(df.shipping_region.unique()):
    top_3_region = obtener_top_k(df[df.shipping_region==region], col='ingresos', k=3)
    print(f'Los ids de los top {len(top_3_region)} productos que reportan los mayores ingresos para la región {region} son: {top_3_region.index.to_list()}')
    print(f'Ingresos:\n{top_3_region}\n')

Los ids de los top 3 productos que reportan los mayores ingresos para la región East son: [25, 40, 15]
Ingresos:
product_id
25    6577.96
40    5636.79
15    5506.16
Name: ingresos, dtype: float64

Los ids de los top 3 productos que reportan los mayores ingresos para la región North son: [6, 7, 5]
Ingresos:
product_id
6    9345.66
7    6851.63
5    5522.07
Name: ingresos, dtype: float64

Los ids de los top 3 productos que reportan los mayores ingresos para la región South son: [35, 30, 20]
Ingresos:
product_id
35    8514.05
30    7350.54
20    7100.23
Name: ingresos, dtype: float64

Los ids de los top 3 productos que reportan los mayores ingresos para la región West son: [37, 22, 29]
Ingresos:
product_id
37    8942.17
22    7663.11
29    7390.49
Name: ingresos, dtype: float64

